In [1]:
import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
class TSCANToEnd(keras.Model):
    
    def __init__(self, model):
        super().__init__()
        self.inner = model
    
    def call(self, x, training=None):
        return self.inner((x, ops.mean(x, axis=(1, ), keepdims=True)), training=training)

model = TSCANToEnd(TSCAN(TN=True))
model.compile(optimizer='adamw', loss='mse')
model(np.random.random((4, 160, 36, 36, 3)));
model.inner.summary()

Model: "tscan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (640, 34, 34, 32)      │        10,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (640, 15, 15, 64)      │        55,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (4, 34, 34, 32)        │        10,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (4, 15, 15, 64)        │        55,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (4, 34, 34, 1)         │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_5 (Sequential)       │ (4, 15, 15, 1)         │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_6 (Sequential)       │ (640, 17, 17, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_7 (Sequential)       │ (640, 1)               │       401,665 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 532,899 (2.03 MB)

 Trainable params: 532,899 (2.03 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
batch_size = 128

rlap_tape = "/root/ssd_cache/rppg_training_data/rlap_160x36x36_all"

train_rlap = load_datatape(rlap_tape, fold='train', batch=batch_size)
valid_rlap = load_datatape(rlap_tape, fold='val', extended_hr='False', batch=batch_size)

#train_rlap = DatatapeMonitor(train_rlap)
training_set, validation_set = KerasDataset(train_rlap), KerasDataset(valid_rlap)

training_set = training_set.apply_fn(compress_aug)

rlap_160x36x36_all             datatape has been loaded.    24704 items total. fold=train selected
rlap_160x36x36_all             datatape has been loaded.     2944 items total. fold=val&extended_hr=False selected


In [4]:
#model.fit(training_set, validation_data=validation_set, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/tscan.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(model, training_set, validation_set, epochs=20, check_point_path='../weights/tscantn.weights.h5')
model.load_weights('../weights/tscantn.weights.h5')

Training:   0%|          | 0/193 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch   1:  Training loss: 0.692	Validation loss:0.5245	


Training:   0%|          | 0/193 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch  19:  Training loss:0.4333	Validation loss:0.4344	


Training:   0%|          | 0/193 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch  20:  Training loss:0.4318	Validation loss:0.4393	


In [5]:
eval_on_dataset(dataset_H5_mmpd, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCANtn_RLAP_MMPD.h5')
get_metrics('../results/TSCANtn_RLAP_MMPD.h5', dropped='False')

  0%|          | 0/660 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,14.225±0.22,23.49±3.584,16.564±0.257,0.35368
Whole,9.127±0.426,14.231±4.17,10.359±0.452,0.5832


In [6]:
eval_on_dataset(dataset_H5_cohface, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCANtn_RLAP_COHFACE.h5', fps=30)
get_metrics('../results/TSCANtn_RLAP_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,3.253±0.2,9.162±3.02,4.357±0.253,0.73401
Whole,2.536±0.433,6.095±3.814,3.33±0.507,0.85988


In [7]:
eval_on_dataset(dataset_H5_pure, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCANtn_RLAP_PURE.h5')
get_metrics('../results/TSCANtn_RLAP_PURE.h5') 

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,0.548±0.056,1.646±0.932,0.825±0.09,0.99744
Whole,0.281±0.087,0.728±0.626,0.423±0.159,0.99951


In [8]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCANtn_RLAP_UBFC.h5')
get_metrics('../results/TSCANtn_RLAP_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,1.306±0.095,2.522±1.156,1.329±0.096,0.98905
Whole,0.561±0.089,0.806±0.466,0.566±0.086,0.99906
